In [1]:
import requests
import pandas as pd
from ipywidgets import widgets
from IPython.display import display

# Fonction pour vérifier si une adresse IP est un relais Tor
def check_tor_relay(ip_address):
    url = f"https://onionoo.torproject.org/summary?search={ip_address}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data['relays']:
            relays_info = []
            for relay in data['relays']:
                relays_info.append({
                    'Nom': relay['n'],
                    'Fingerprint': relay['f'],
                    'Adresses': ', '.join(relay['a']),
                })
            return relays_info
        else:
            return None
    else:
        return [{'Nom': 'Erreur', 'Fingerprint': 'Erreur', 'Adresses': f"Erreur lors de la requête: {response.status_code}"}]

# Interface pour copier-coller des adresses IP
ip_input_area = widgets.Textarea(
    value='',
    placeholder='Collez ici les adresses IP, une par ligne',
    description='Adresses IP:',
    layout=widgets.Layout(width='100%', height='200px')
)

run_button = widgets.Button(description="Vérifier les IPs")
output = widgets.Output()

def on_run_button_clicked(b):
    with output:
        output.clear_output()
        ip_input = ip_input_area.value.strip()
        if ip_input:
            ip_list = ip_input.split('\n')
            results = []
            negative_results = []

            for ip in ip_list:
                result = check_tor_relay(ip.strip())
                if result is None:
                    negative_results.append(f"L'IP {ip} n'est pas un nœud Tor actuellement.")
                else:
                    results.extend(result)

            # Affichage des résultats
            if results:
                results_df = pd.DataFrame(results)
                display(results_df)
            else:
                print("Aucun relais Tor trouvé.")

            if negative_results:
                print("Résultats négatifs :")
                for neg_res in negative_results:
                    print(neg_res)

run_button.on_click(on_run_button_clicked)

display(ip_input_area, run_button, output)


Textarea(value='', description='Adresses IP:', layout=Layout(height='200px', width='100%'), placeholder='Colle…

Button(description='Vérifier les IPs', style=ButtonStyle())

Output()